## 基础题目1

切分数据，要求如下：

+ 针对每一 target，分别按 80%/20% 将数据切分为训练数据和测试数据
+ 切分时要求每 5 个样本，取中间的 1 个为测试数据，其余为训练数据


## 基础题目2
预测结果，要求如下：

+ 加载预先在训练数据上训练好的权重参数（存储参数的方法：np.savez("data/weight", weight=weight)）
+ 使用模型 sigmoid(W·X) 对测试数据进行预测
+ 输出预测的准确率

## 进阶题目
如果您已经对机器学习有了一定基础，不妨试试这个练习，它会使用更多 numpy 的 API。

使用 Numpy 实现简单神经网络，要求如下：

+ 特征选择第 2 个和第 4 个（sepal width, petal width）
+ 使用反向传播和梯度下降进行学习

In [19]:
import numpy as np
from sklearn.datasets import load_iris
np.random.seed(12345)

In [20]:
# 加载 iris 数据集
iris = load_iris()
# 提取 target ≤ 1 的 target，并将维度从 (length, ) 拓展为 (length, 1)
y = np.expand_dims(np.extract(iris["target"] <= 1, iris["target"]), 1)
# 根据 y 的 size 拿到对应的 x
x = iris["data"][:y.shape[0]]

In [21]:
y.shape, x.shape

((100, 1), (100, 4))

In [33]:
# 有多少条数据
data_size = x.shape[0]
# 20% 作为测试数据
test_size = int(data_size * 0.2)
# 找到测试数据的 index
test_idx = [i*5 for i in range(test_size)]
# 过滤掉测试数据的 index，剩下的就是训练数据的 index
train_idx = [i for i in range(data_size) if i not in test_idx]
len(test_idx), len(train_idx)

(20, 80)

In [34]:
# 使用训练数据的 index 得到训练数据
x_train = x[train_idx]
y_train = y[train_idx]
# 直接使用索引切片得到测试数据，当然也可以使用测试数据 index
x_test = x[0: data_size : 5]
y_test = y[0: data_size : 5]

- 基础题目2

In [39]:
# Sigmoid
def sigmoid(x: np.array, derive: bool = False) -> np.array:
    if derive:
        return x * (1 - x)
    return 1 / (1 + np.exp(-x))

#预测函数
def predict(x, w):
    return sigmoid(x.dot(w))

In [44]:
# 加载权重，注意 key="weight"
w = np.load("./data/weight.npz")["weight"]
w.shape

(4, 1)

In [42]:
# 拿到 sigmoid 的结果，即 probability
prob = predict(x_test, w)


In [45]:
# >（或 ≥）0.5 作为 1 或 0 的标准
sum((prob > 0.5) == y_test) / len(y_test)

array([1.])

- 进阶题目

In [53]:
# 只取第2和第4个特征
x_train_part = x_train[:, [1, 3]]
x_test_part = x_test[:, [1, 3]]

(20, 2)

In [47]:
def test(x, y, wh, wo, use_hidden: bool = False):
    """
    预测函数，如果使用隐层的话，需要增加对应的计算
    """
    if use_hidden:
        ypred = sigmoid(sigmoid(x.dot(wh)).dot(wo))
    else:
        ypred = sigmoid(x.dot(wo))
    print("Acc: %.2f" % (sum((ypred >= 0.5 ) == y) / len(y))[0])

In [71]:
def train(x, y, use_hidden: bool = False, lr: float = 0.01):
    """
    训练函数，可以支持单隐层或无隐层
    """
    hidden_size = 8 if use_hidden else x.shape[1]
    # 初始化，可以有多种初始化方式，我们这里使用 uniform
    wh = np.random.uniform(-np.sqrt(1/hidden_size), np.sqrt(1/hidden_size), (x.shape[1], hidden_size))
    wo = np.random.uniform(-1, 1, (hidden_size, 1))
    # 1000 个 Epoch
    for epoch in range(1, 1001):
        # 使用隐层
        if use_hidden:
            hidden = np.dot(x, wh)
            hidden = sigmoid(hidden)
        else:
            hidden = x
        logits = np.dot(hidden, wo)
        pred = sigmoid(logits)
        # 输出层的 error
        pred_err = y - pred
        # 反向传播，输出层的更新权重
        pred_delta = pred_err * sigmoid(pred, True)
        if use_hidden:
            # 隐层的 error 和更新权重
            hidd_err = pred_delta.dot(wo.T)
            hidd_delta = hidd_err * sigmoid(hidden, True)
        
        # 更新参数
        wo += lr * hidden.T.dot(pred_delta)
        if use_hidden:
            wh += lr * x.T.dot(hidd_delta)

        # 计算损失
        loss = np.mean(np.abs(pred_err))
        if epoch % 200 == 0:
            # 如果有验证集，可以在这里对验证集进行验证，结果有提升（如 loss下降或 acc 提高）时进行存储
            # 还可以根据结果提升情况设计自动提前终止的方案（比如连续 3 次不再提升）
            print("Error:" + str(loss))
    
    # 得到训练集的 Acc
    test(x, y, wh, wo, use_hidden)
    # 返回权重（参数）
    return wh, wo

In [72]:
wh, wo = train(x_train_part, y_train)
wh.shape, wo.shape

Error:0.12356083588449022
Error:0.08331958773616781
Error:0.06628462513391868
Error:0.05637309253951559
Error:0.049715585633962125
Acc: 1.00


((2, 2), (2, 1))

In [73]:
test(x_test_part, y_test, wh, wo)

Acc: 1.00
